# Preprocess Input Data and Create processed Json file

---

---

# 1. Set up

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
# module_path = "../../../utils"
# sys.path.append(os.path.abspath(module_path))
# print(os.path.abspath(module_path))

module_path = "./utils"
sys.path.append(os.path.abspath(module_path))
print(os.path.abspath(module_path))

/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs/utils


# 2. Remvoe duplicated contents 

In [2]:
from utils.proc_docs import get_load_json, show_doc_json

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
import glob
import pandas as pd

# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/*.json'
json_files = glob.glob(folder_path)

def remove_duplicate_col(json_files, remove_col):
    '''
    json_files : list of json file path
    remove_col : column name to remove duplicate
    Desc: 
    '''
    all_df = pd.DataFrame()
    
    for file_path in json_files:
        df = pd.read_json(file_path)
        df = pd.json_normalize(df.sections) # Normalize sections
        all_df = pd.concat([all_df, df])
    
    all_df.drop_duplicates(subset=remove_col, inplace=True)
    
    return all_df

df = remove_duplicate_col(json_files, remove_col='content')
df

,title,last_updated,content,url,project
0,Get support,2023-07-26,Get support. To get support Select Support fro...,https://docs.samsungknox.com/admin/knox-guard/...,KG
1,How-to videos,2023-10-17,How-to videos. This section contains a video i...,https://docs.samsungknox.com/admin/knox-guard/...,KG
2,Knox Guard,2023-07-26,Knox Guard. Knox Guard is a cloud-based servic...,https://docs.samsungknox.com/admin/knox-guard,KG
3,Access the Knox Guard console,2023-07-26,Access the Knox Guard console. To access the K...,https://docs.samsungknox.com/admin/knox-guard/...,KG
4,Knox Guard status flow,2023-07-26,Knox Guard status flow. Once a device is added...,https://docs.samsungknox.com/admin/knox-guard/...,KG
...,...,...,...,...,...
16,What license do I need for Knox Authentication...,2023-07-26,What license do I need for Knox Authentication...,https://docs.samsungknox.com/admin/knox-authen...,KAM
17,How can I tell if my license is expired?,2023-07-26,How can I tell if my license is expired?. If y...,https://docs.samsungknox.com/admin/knox-authen...,KAM
18,Why do I get a license activation failed error?,2023-07-26,Why do I get a license activation failed error...,https://docs.samsungknox.com/admin/knox-authen...,KAM
19,Unable to authenticate face after v1.3 update,2023-09-26,Unable to authenticate face after v1.3 update....,https://docs.samsungknox.com/admin/knox-authen...,KAM


# 3. Craete new json file

In [4]:
cleaned_json_file_path = "data/poc/preprocessed_json/all_processed_data.json"
df.reset_index(drop=True).to_json(cleaned_json_file_path,orient='records')

Verify the created json file

In [5]:
df_json = pd.read_json(cleaned_json_file_path)
df_json.head(3)

,title,last_updated,content,url,project
0,Get support,2023-07-26,Get support. To get support Select Support fro...,https://docs.samsungknox.com/admin/knox-guard/...,KG
1,How-to videos,2023-10-17,How-to videos. This section contains a video i...,https://docs.samsungknox.com/admin/knox-guard/...,KG
2,Knox Guard,2023-07-26,Knox Guard. Knox Guard is a cloud-based servic...,https://docs.samsungknox.com/admin/knox-guard,KG


In [6]:
%store cleaned_json_file_path

Stored 'cleaned_json_file_path' (str)


# 4. Translate English to Korean

In [7]:
import boto3
translate_client = boto3.client(service_name='translate', region_name='us-west-2', use_ssl=True)

## Text Splitter 정의 
- AWS Translate 는 최대 10000 bytes 를 번역할 수 있어서, 조금 더 작은 단위로 문서를 쪼개서 번역 함

In [8]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, SpacyTextSplitter

chunk_size = 8192
chunk_overlap = 0

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

In [9]:
%%time

import pandas as pd

def translate_df(df, target_col, new_col):
    
    def run_translate(text):
        '''
        Split doc into small chunks, translate them, and then merge into one
        '''
        doc = [Document(page_content=text)]
        docs = text_splitter.split_documents(doc)                
        
        translate_doc = []
        for i, chunk in enumerate(docs):
            # print("English Text: \n", chunk.page_content)                    
            result = translate_client.translate_text(Text=chunk.page_content, 
                    SourceLanguageCode="en", TargetLanguageCode="ko")
            result = result['TranslatedText']
            translate_doc.append(result)
        # print("# of chunks: ", i)
            
        translate_doc = ' '.join(translate_doc)            
        # print("translate_doc: \n", translate_doc)
        
        return translate_doc
    
    def translate(df):
        text = df[target_col]
        result = run_translate(text)
        # print(result)        

        return result
        
    # Start here
    df[new_col] = df.apply(translate, axis=1)
    
    return df




CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 9.78 µs


## 번역 실행

In [10]:
%%time
df_kr = translate_df(df, target_col='title', new_col='title_ko')
df_kr = translate_df(df_kr, target_col='content', new_col='content_ko')


CPU times: user 6.93 s, sys: 224 ms, total: 7.16 s
Wall time: 10min 28s


In [11]:
def post_process_translate(df):
    df.drop(columns=['title', 'content'], inplace=True)
    df.rename(columns={'title_ko': 'title', 'content_ko': 'content'}, inplace=True)

    return df

df_post = post_process_translate(df_kr)
df_post

,last_updated,url,project,title,content
0,2023-07-26,https://docs.samsungknox.com/admin/knox-guard/...,KG,지원 받기,지원 받기.지원을 받으려면 왼쪽 탐색 메뉴에서 지원을 선택하여 Knox Guard에...
1,2023-10-17,https://docs.samsungknox.com/admin/knox-guard/...,KG,사용 방법 동영상,사용 방법 동영상.이 섹션에는 Knox Guard를 소개하는 비디오가 포함되어 있습...
2,2023-07-26,https://docs.samsungknox.com/admin/knox-guard,KG,녹스 가드,"녹스 가드.Knox Guard는 사기와 도난으로부터 디바이스, 기업 자산, 결제 플..."
3,2023-07-26,https://docs.samsungknox.com/admin/knox-guard/...,KG,녹스 가드 콘솔에 액세스,녹스 가드 콘솔에 액세스하세요.Knox Guard 콘솔에 액세스하려면 먼저 등록하고...
4,2023-07-26,https://docs.samsungknox.com/admin/knox-guard/...,KG,녹스 가드 상태 흐름,녹스 가드 상태 흐름.디바이스가 Knox Guard 콘솔에 추가되면 잠재적으로 다음...
...,...,...,...,...,...
16,2023-07-26,https://docs.samsungknox.com/admin/knox-authen...,KAM,Knox 인증 관리자를 사용하려면 어떤 라이선스가 필요한가요?,Knox 인증 관리자를 사용하려면 어떤 라이선스가 필요한가요?Knox 인증 관리자를...
17,2023-07-26,https://docs.samsungknox.com/admin/knox-authen...,KAM,라이선스가 만료되었는지 어떻게 알 수 있나요?,라이선스가 만료되었는지 어떻게 알 수 있나요?.라이선스가 만료된 경우 디바이스 사용...
18,2023-07-26,https://docs.samsungknox.com/admin/knox-authen...,KAM,라이선스 활성화 실패 오류가 발생하는 이유는 무엇인가요?,라이선스 활성화 실패 오류가 발생하는 이유는 무엇인가요?Knox Suite 라이선스...
19,2023-09-26,https://docs.samsungknox.com/admin/knox-authen...,KAM,v1.3 업데이트 이후 얼굴을 인증할 수 없음,"v1.3 업데이트 이후 얼굴을 인증할 수 없습니다.환경, Knox Authentic..."


# 5. Craete new korean Json file

In [ ]:
def create_json_file(df, file_name):
    df.to_json(file_name, orient='records', force_ascii=False)
    print(f"{file_name} is generated")

cleaned_json_file_path = "data/poc/preprocessed_json/all_processed_data_ko.json"    
create_json_file(df_post, file_name="data/poc/preprocessed_json/all_processed_data_ko.json")


In [ ]:
df_json = pd.read_json(cleaned_json_file_path)
df_json.head(3)